# 智能床使用前后入睡潜伏期数据处理

In [1]:
!pip install python-docx

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import os
import pandas as pd
import re
from docx import Document

In [3]:
#判断小时位上是否小于12h，如果小于12小时加一天
def time_datetime(time):
    l = []
    for i in time:

        t = int(i.split(":")[0])
        if t <12:
            date = "2024-01-02"
        else:
            date = "2024-01-01"
        l.append(f'{date}-{i}')

    return l

In [4]:
# 差值转换成分钟
def total_time(x1,x2):
    minutess = []
    x1 = pd.to_datetime(x1)
    x2 = pd.to_datetime(x2)

    sleep_time = (x2-x1)
    for i in range(len(sleep_time)):
        m = int((sleep_time[i].total_seconds())//60)
        minutess.append(int(m))
    return minutess

In [5]:
path = "../data/第2晚已监测完"

In [6]:
#计算智能床使用前、后第一帧睡眠时间开始时间
dir = os.listdir(path)
list_name = []
list_before =[]
list_after = [] 
for i in dir:
    name = str(i)
    name = re.findall(r'[（](.*?)[）]',name)
    smart_bed_beofre = os.path.join(path,i,"Before_Use/Smart_Bed_Data/Sleep_Sign_Report_Data/Sleep_Report.csv")
    smart_bed_after = os.path.join(path,i,"After_Use/Smart_Bed_Data/Sleep_Sign_Report_Data/Sleep_Report.csv")
    df_before = pd.read_csv(smart_bed_beofre)

    df_after = pd.read_csv(smart_bed_after)
    
    before_bedtime = df_before["col_sleep_time"][0][-8:-3]

    after_bedtime = df_after["col_sleep_time"][0][-8:-3]

    list_name.append(name[0])
    #智能床使用前第一帧睡眠时间开始时间
    list_before.append(before_bedtime)
    #智能床使用后第一帧睡眠时间开始时间
    list_after.append(after_bedtime)


dir = os.listdir(path)
list_light_before= []
list_light_after = []
s_before = []
s_after = []

In [7]:
for i in dir:
    pathdir = os.path.join(path,i)
    name = re.findall(r'[（].*?[）]',str(i))
    name = name[0]

    before = os.path.join(pathdir,"Before_Use/Psg_Data/Hsy_Sleep_Report/Hsy_Sleep_Report1.docx")
    after = os.path.join(pathdir,"After_Use/Psg_Data/Hsy_Sleep_Report/Hsy_Sleep_Report1.docx")
    document = Document(before)  # 读入文件

    # psg使用前关灯时间
    light_time = document.tables[1].rows[0].cells[0].text
    ligtht = light_time[-5:]
    list_light_before.append(ligtht)
    
    # psg使用后关灯时间
    document = Document(after)
    light_time = document.tables[1].rows[0].cells[0].text
    ligtht = light_time[-5:]
    list_light_after.append(ligtht)
    

    # 计算psg使用前入睡潜伏期
    psg_before =  os.path.join(pathdir,"Before_Use/Psg_Data/Hsy_Sleep_Report/Hsy_Sleep_Report1.docx")
     
    document = Document(psg_before)

    psg_before_text = (document.tables[1].rows[0].cells[0].text)[-5:]
    light_time = document.tables[1].rows[1].cells[2].text

    sleep_time = light_time[6:-1]
    s_before.append(sleep_time)

    # 计算psg使用后入睡潜伏期
    psg_after =  os.path.join(pathdir,"After_Use/Psg_Data/Hsy_Sleep_Report/Hsy_Sleep_Report1.docx")

    document = Document(psg_after)

    psg_after_text = (document.tables[1].rows[0].cells[0].text)[-5:]

    light_time = document.tables[1].rows[1].cells[2].text
    sleep_time = light_time[6:-1]
    s_after.append(sleep_time)


In [8]:
# 写成dataframe格式
df = pd.DataFrame()
df["姓名"] =list_name
df["psg使用前关灯时间"] = list_light_before
df['智能床使用前睡眠开始'] = list_before
df ["psg使用后关灯时间"] =list_light_after
df['智能床使用后睡眠开始'] = list_after
df['psg使用前入睡潜伏期'] = s_before
df['psg使用后入睡潜伏期'] = s_after

In [9]:
#计算使用前智能床入睡潜伏期（智能床使用前第一帧入睡时间-PSG使用前关灯时间）
x1 = time_datetime(df['psg使用前关灯时间'])
x2 = time_datetime(df['智能床使用前睡眠开始'])

y1 = total_time(x1,x2)
df["使用前相差"] =y1

#计算使用后智能床入睡潜伏期（智能床使用后第一帧入睡时间-PSG使用后关灯时间）
x1 = time_datetime(df['psg使用后关灯时间'])
x2 = time_datetime(df['智能床使用后睡眠开始'])
y1 = total_time(x1,x2)
df["使用后相差"] =y1

df.to_excel("../result/智能床完整入睡潜伏期.xlsx")